In [1]:
import logging
from ome_zarr_utils import (
    convert_czi2hcs_omezarr,
    convert_czi2hcs_ngff,
    omezarr_package,
    setup_logging,
)
import ngff_zarr as nz
from pathlib import Path

In [2]:
# ========== Configuration Parameters ==========
# Toggle to display the result in napari viewer (requires napari installation)
show_napari: bool = False

# Backend library selection: OME_ZARR (ome-zarr-py) or NGFF_ZARR (ngff-zarr)
ome_package = omezarr_package.OME_ZARR
# ome_package = omezarr_package.NGFF_ZARR

# Scene ID for non-HCS format (ignored if write_hcs=True)
scene_id: int = 0

# Use absolute path to external test data
filepath: str = r"F:\Github\omezarr_playground\data\WP96_4Pos_B4-10_DAPI.czi"

# ========== Setup Logging ==========
czi_path = Path(filepath)
log_file_path = czi_path.parent / f"{czi_path.stem}_conversion.log"

# Configure logging explicitly
setup_logging(str(log_file_path))
logger = logging.getLogger(__name__)

In [3]:
logger.info(f"Converting CZI to HCS-ZARR format using {ome_package.name}...")

if ome_package == omezarr_package.OME_ZARR:
    # Convert using ome-zarr-py backend
    zarr_output_path = convert_czi2hcs_omezarr(filepath, overwrite=True)

elif ome_package == omezarr_package.NGFF_ZARR:
    # Convert using ngff-zarr backend
    zarr_output_path = convert_czi2hcs_ngff(filepath, overwrite=True)
else:
    raise ValueError(f"Unsupported ome_package: {ome_package}")

logger.info(f"✓ Converted to OME-ZARR HCS format at: {zarr_output_path}")

2025-11-13 15:27:53,876 - __main__ - INFO - Converting CZI to HCS-ZARR format using OME_ZARR...
2025-11-13 15:27:53,878 - ome_zarr_utils - INFO - ================================================================================
2025-11-13 15:27:53,879 - ome_zarr_utils - INFO - CZI to HCS OME-ZARR Conversion Started (OME-ZARR format)
2025-11-13 15:27:53,880 - ome_zarr_utils - INFO - ================================================================================
2025-11-13 15:27:53,880 - ome_zarr_utils - INFO - Input CZI file: F:\Github\omezarr_playground\data\WP96_4Pos_B4-10_DAPI.czi


Reading 2D planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

2025-11-13 15:27:54,288 - ome_zarr_utils - INFO - Writing Well: B/10, Field: 0, Scene Index: 24
2025-11-13 15:27:54,360 - ome_zarr_utils - INFO - Writing Well: B/10, Field: 1, Scene Index: 25
2025-11-13 15:27:54,418 - ome_zarr_utils - INFO - Writing Well: B/10, Field: 2, Scene Index: 26
2025-11-13 15:27:54,481 - ome_zarr_utils - INFO - Writing Well: B/10, Field: 3, Scene Index: 27
2025-11-13 15:27:54,597 - ome_zarr_utils - INFO - Writing Well: B/4, Field: 0, Scene Index: 0
2025-11-13 15:27:54,661 - ome_zarr_utils - INFO - Writing Well: B/4, Field: 1, Scene Index: 1
2025-11-13 15:27:54,717 - ome_zarr_utils - INFO - Writing Well: B/4, Field: 2, Scene Index: 2
2025-11-13 15:27:54,772 - ome_zarr_utils - INFO - Writing Well: B/4, Field: 3, Scene Index: 3
2025-11-13 15:27:54,834 - ome_zarr_utils - INFO - Writing Well: B/5, Field: 0, Scene Index: 4
2025-11-13 15:27:54,898 - ome_zarr_utils - INFO - Writing Well: B/5, Field: 1, Scene Index: 5
2025-11-13 15:27:54,963 - ome_zarr_utils - INFO - Wr

In [4]:
# Validate the HCS-ZARR file against OME-NGFF specification
# This ensures proper metadata structure for multi-well plate data

logger.info("Validating created HCS-ZARR file against schema...")
zarr_output_path = nz.from_hcs_zarr(zarr_output_path, validate=True)
logger.info("✓ Validation successful - HCS metadata conforms to specification.")

2025-11-13 15:27:56,200 - __main__ - INFO - Validating created HCS-ZARR file against schema...
2025-11-13 15:27:56,300 - __main__ - INFO - ✓ Validation successful - HCS metadata conforms to specification.


In [5]:
# Optionally display the result in napari viewer

if show_napari:
    import napari

    # show in napari (requires napari to be installed!)
    viewer = napari.Viewer()
    viewer.open(zarr_output_path, plugin="napari-ome-zarr")

In [6]:
# Capture screenshot of the napari viewer if displayed

if show_napari:
    napari.utils.nbscreenshot(viewer)